In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp ~/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c ml-intensive-yandex-autumn-2023

In [ ]:
! unzip /content/ml-intensive-yandex-autumn-2023.zip -d /root
! mkdir /root/model1
! mkdir /root/model2
! mkdir /root/data/slice_train_images

In [ ]:
import keras
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras import layers
from sklearn.metrics import f1_score
import os

#URLы для картинок
train_url = '/root/data/train_images/'
train_mask_url = '/root/data/train_lung_masks/'
train_label_url = '/root/data/train_answers.csv'
test_url = '/root/data/test_images/'
slice_train_url = '/root/data/slice_train_images'
# URL куда я сохранял модель
save_model = '/root/model1/'
# загрузка имен картинок и их обрезание до чисел, понадобится при сохранении результатов предсказания модели в CSV
path = os.listdir('/root/data/test_images')
path = [int(i[4:-4]) for i in path]
'''
# для перевода картинок в массивы Numpy
image_list = os.listdir(train_url)
train_list = []
mask_list = []
for i in image_list:
    image = Image.open(train_url+'\\'+i)
    image = np.asarray(image)
    train_list.append(image)
    image = Image.open(train_mask_url+'\\'+i)
    image = np.asarray(image)
    mask_list.append(image)
print('download complete')
# перевод списка в массив Numpy
train_list = np.array(train_list)
# перевод маски в массив нулей и единиц для умножения
mask_list = np.array(mask_list)//255
# выделение легких по маскам
slice_list = train_list*mask_list
print('preprocess complete')
# сохранение обрезанных легких
for i in range(len(slice_list)):
    image = Image.fromarray(slice_list[i],mode='L')
    image.save(slice_train_url+'\\'+image_list[i])
print('Save complete')
#просмотр вырезанных легких
plt.figure(figsize=(15,8))
for i in range(5):
    plt.subplot(2,5,1+i)
    plt.imshow(train_list[i])
    plt.xticks([])
    plt.yticks([])
    plt.subplot(2,5,6+i)
    plt.imshow(slice_list[i])
    plt.xticks([])
    plt.yticks([])
plt.show()
'''
# блок с переменными
EPOCHS = 50
batch_size = 20
input_shape = (128, 128, 1)
# чтение меток
train_label = pd.read_csv(train_label_url)
# перевод id в строку для сопоставления сортировки с названиями картинок
train_label['id'] = train_label['id'].astype('string')
train_label = train_label.sort_values(by='id', ascending=True)
# переводим метки в список
train_label = list(train_label['target_feature']>0)
# загрузка и разделение на train и valid,
# color_mode - формат картинок в сером стиле(одно измерение)
# validation_split- разделение 27000 на 24к и 3к
# subset- для указания какой будет датасет (train, valid или both-оба)
# image_size - размер картинок при импорте,
# забыл сказать что при уменьшении размера они еще и памяти жрут в 2 раза меньше, что прям хорошо
train_set, valid_set = keras.utils.image_dataset_from_directory(train_url, labels=train_label, batch_size=batch_size,
                                                                shuffle=False, color_mode='grayscale',
                                                                validation_split=1 / 9, subset='both',
                                                                image_size=(128, 128))
# выделение меток из валидационного сета,
# пришлось выделять из датасета что бы не потерялось сопоставление картинок и меток
b = valid_set.map(lambda x,y:y)
b = np.array([i.numpy() for i in b])
b = b.reshape(3000)
# загрузка тестового датасета
test_set = keras.utils.image_dataset_from_directory(test_url, labels=None, image_size=(128, 128),
                                                    color_mode='grayscale',shuffle=False)

'''
# визуализация датасетов
class_names = ['not_sick', 'not_covid', 'covid']
plt.figure(figsize=(15, 8))
for i in train_set.take(1):
    for j in range(20):
        plt.subplot(4, 5, 1 + j)
        plt.imshow(i[0][j].numpy() / 255)
        plt.title(int(i[1][j]))
        plt.xticks([])
        plt.yticks([])
plt.grid(False)
plt.savefig('set1.png')
'''
# аугментационная модель
aug = keras.Sequential([
    layers.RandomFlip(seed=42),
    layers.RandomContrast(factor=0.1, seed=42),
    layers.RandomBrightness(factor=0.1, value_range=(0, 255), seed=42),
    layers.RandomRotation(factor=0.1, seed=42)
])
# обработка датасета в отдельную переменную, что бы потом их можно было сложить в один датасет
train_set2 = train_set.map(lambda x, y: (aug(x), y))

'''
# визуализация аугментированных картинок
plt.figure(figsize=(15, 8))
for i in train_set2.take(1):
    for j in range(20):
        plt.subplot(4, 5, 1 + j)
        plt.imshow(i[0][j].numpy() / 255)
        plt.title(int(i[1][j]))
        plt.xticks([])
        plt.yticks([])
plt.grid(False)
plt.savefig('set2.png')
'''
# объединение аугментации с обычным датасетом и удаление аугментационной переменной для экономии оперативки
train_set = train_set.concatenate(train_set2)
del train_set2

# функция для создания моделей
def build_model(x=1):
    if x == 1:
        mod = tf.keras.Sequential([
            layers.Rescaling(1 / 255),
            layers.Input(input_shape),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Flatten(),
            layers.Dense(600, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(30, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
    elif x == 2:
        mod = tf.keras.Sequential([
            layers.Rescaling(1 / 255),
            layers.Input(input_shape),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2,2)),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.5),
            layers.AvgPool2D((2, 2)),
            layers.Flatten(),
            layers.Dense(30, activation='relu'),
            layers.Dense(30, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
    elif x == 3:
        mod = tf.keras.Sequential([
            layers.Rescaling(1 / 255),
            layers.Input(input_shape),
            layers.Conv2D(32, 3, activation='relu', padding='same', strides=1),
            layers.Dropout(0.4),
            layers.AvgPool2D((2,2)),
            layers.Flatten(),
            layers.Dense(300, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(200, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(100, activation='relu'),
            layers.Dropout(0.1),
            layers.Dense(3, activation='softmax')
        ])
    return mod


model = build_model(1)
# здесь думаю и так все понятно)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"]
              )
# callback - выполняет действие при каком то условии,
# в данном случае сохраняет модель после эпохи,
# если на этой эпохе получен лучший результат, чем ранее
callbacks = keras.callbacks.ModelCheckpoint(
    save_model,
    # monitor - то за чем следит модель, можно указать val_loss или train_accuracy
    monitor="val_accuracy",
    verbose=1, # сообщает когда выполнен callback 0-не сообщать
    save_best_only=True, # сохранять лучшую модель
    save_weights_only=True, # сохранять только весы(можно целиком, но тогда модель будет переписана полностью,
    # и будет как в момент сохранения)
    mode="auto", # ориентация для monitor, может быть max ,min , auto (например max val_loss-будет сохранять модель
    # при увеличении лосса
    save_freq="epoch", # когда сохранять
    initial_value_threshold=None, # начальное значение для перезаписи(например если задать 0.9,
    # перепишет модель только если monitor будет выше/ниже этого значения)
)
# загрузка весов из модели, лучше закомментить в первый раз или если изменил параметры модели, иначе выдаст ошибку
#model.load_weights(save_model)
# запуск обучения
history = model.fit(train_set, epochs=EPOCHS, validation_data=valid_set, shuffle=True, callbacks=callbacks)
# эти строки для сравнения accurracy с f1-score,сами увидите что результаты почти не отличаются.
check = model.predict(valid_set)
check = tf.argmax(check, axis=1)
check = f1_score(check, b,average='micro')
print(check)
# собственно предсказание по тестовой выборке с текущей моделью
ans = model.predict(test_set)
ans = tf.argmax(ans, axis=1)
# перевод в pd.DataFrame  и сохранение
ans = pd.DataFrame(data=ans, columns=['target_feature'])
ans['id'] = path
ans = ans.sort_values('id')
ans.to_csv('ans1.csv', encoding='utf-8',index=False)
# теперь загружаем лучшую модель и повторяем предсказание на тестовой выборке
model.load_weights(save_model)
ans = model.predict(test_set)
ans = tf.argmax(ans, axis=1)
ans = pd.DataFrame(data=ans, columns=['target_feature'])
ans['id'] = path
ans = ans.sort_values('id')
ans.to_csv('ans2.csv', encoding='utf-8',index=False)
# просмотр графиков
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('score')
plt.legend(loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('score')
plt.legend(loc='lower right')

Found 27000 files belonging to 2 classes.
Using 24000 files for training.
Using 3000 files for validation.
Found 6920 files belonging to 1 classes.
Epoch 1/50
2399/2400 [============================>.] - ETA: 0s - loss: 0.4989 - accuracy: 0.7647
Epoch 1: val_accuracy improved from -inf to 0.83133, saving model to /root/model/
2400/2400 [==============================] - 110s 45ms/step - loss: 0.4990 - accuracy: 0.7647 - val_loss: 0.3972 - val_accuracy: 0.8313
Epoch 2/50
2399/2400 [============================>.] - ETA: 0s - loss: 0.4181 - accuracy: 0.8042
Epoch 2: val_accuracy improved from 0.83133 to 0.85767, saving model to /root/model/
2400/2400 [==============================] - 107s 45ms/step - loss: 0.4182 - accuracy: 0.8041 - val_loss: 0.3466 - val_accuracy: 0.8577
Epoch 3/50
2400/2400 [==============================] - ETA: 0s - loss: 0.3987 - accuracy: 0.8166
Epoch 3: val_accuracy did not improve from 0.85767
2400/2400 [==============================] - 102s 43ms/step - loss: 